In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import sys
import re
import logging
import os
import re
from pathlib import Path
from functools import partial, cache, cached_property
from itertools import chain
from inspect import signature
from uuid import uuid4

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
import talib as ta 
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer, BarRenderer, Expr, VegaExpr
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import ParameterGrid
from termcolor import colored
from joblib import Parallel, delayed
# from bhsysapi import BHSysApi

from techi import api as ti

from gioutils import ezutils as ez
from gioutils.ezutils import bh, bhs
from gioutils.blpw import BlpQuery
from gioutils.gui.pandas import millify
from gioutils.utils import (
    RepeatedTimer, 
    parse_swap_tenor_expr, 
    parse_offset,
    today, 
    get_next_n, 
    get_n_from_code,
    date_from_offset, 
    get_bday,
    get_bbg_fut_chain_ticker
)

from api.data.base import data_path, radar_path
from api.gui.params import bbg_params, spot_params, fwd_params
from api.instruments.swap import RealSwap, RollingSwap, Fly, Spread
from api.instruments.utils import parse_dates, parse_number, ez_bump
from api.instruments.strategies import FutureStrategy
from api.instruments.signals import MACDHistCrossover, ZScoreThreshold, EnsambleSignal
from api.instruments.cache import data_cache
from api.instruments.future import FutureChain, FutureSpread, FutureFly
from api.gui.theme import bg_color, grid_style
from api.gui.pandas import DFOutput, negative_red, custom_styler_row_line, DataGridGio
from api.instruments import config as c
from api.instruments.signals2 import BaseSignal, MACDCustomSignal, get_crossover_signal
from api.instruments.strategies2 import MACDCustomStrategy

In [3]:
pd.options.plotting.backend = "plotly"
pd.options.display.float_format = millify
c.USE_T_COST = True

In [4]:
bq = BlpQuery(timeout=50000).start()
# ba = BHSysApi(password="A&3jg5Aj")

In [5]:
notional = 1000
hist = '10y'
strategy_hist = '1y'

## Instruments Parameters

In [6]:
fx = [
    'EURUSD',
    'USDJPY',
    'EURGBP',
    'USDGBP'
]

In [7]:
steps = dict(
    fastperiod=1,
    slowperiod=1,
    signalperiod=1,
    time_barrier=1,
    q=0.1,
    k=0.1,
    crossover_days=1,
    signal_threshold_buy=10,
    signal_threshold_sell=10,
    base_stop_periods=1,
    base_stop_mult=1,
)

In [8]:
macd_params = {
    'fastperiod': np.arange(12, 12 + steps['fastperiod'], steps['fastperiod']).tolist(),
    'slowperiod': np.arange(26, 26 + steps['slowperiod'], steps['slowperiod']).tolist(),
    'signalperiod': np.arange(9, 9 + steps['signalperiod'], steps['signalperiod']).tolist(),
}

strategy_params = {
    'time_barrier': np.arange(4, 5 + steps['time_barrier'], steps['time_barrier']).tolist(),
    'q': np.arange(0.4, 0.9 + steps['q'], steps['q']).round(int(np.abs(np.log10(steps['q'])))).tolist(),
    'k': np.arange(2, 3 + steps['k'], steps['k']).round(int(np.abs(np.log10(steps['k'])))).tolist(),
    'crossover_days': np.arange(3,  3 + steps['crossover_days'], steps['crossover_days']).tolist(),
    'base_stop_periods': np.arange(7,  7 + steps['base_stop_periods'], steps['base_stop_periods']).tolist(),
    'base_stop_mult': np.arange(2,  2 + steps['base_stop_mult'], steps['base_stop_mult']).tolist(),
}

In [9]:
macd_dynamic_params = [
    'signal_threshold_buy',
    'signal_threshold_sell',
    'slope_threshold_buy',
    'slope_threshold_sell',
]

In [10]:
macd_param_grid = ParameterGrid(macd_params)

In [11]:
signal_threshold_quantile1 = .75
signal_threshold_quantile2 = .99
slope_threshold_quantiles = [
    0.3, 
    # 0.4, 
    # 0.5
]

In [12]:
def series_fn(instrument):
    return 100 - instrument.yld

In [13]:
from api.instruments.spot import FX

In [14]:
f = FX('USDJPY')

In [15]:
f.quote

date
2022-01-26   114.35
2022-01-27   115.38
2022-01-28   115.26
2022-01-31   115.13
2022-02-01   114.69
              ...  
2023-01-20   129.60
2023-01-23   130.64
2023-01-24   130.02
2023-01-25   129.61
2023-01-26   129.91
Name: USDJPY quote, Length: 262, dtype: float64

### FX

In [39]:
def make_param_grids(
    instrument,
    macd_param_grid, 
    signal_threshold_quantile1=.75,
    signal_threshold_quantile2=.99,
    slope_threshold_quantiles=None,
    notional=1000,
    start_hist='10y',
):
    slope_threshold_quantiles = slope_threshold_quantiles or [
        0.3, 
        # 0.4, 
        # 0.5
    ]
    instrument_param_grids = []
    for macd_param in macd_param_grid:
        signal = MACDCustomSignal(ti_kwargs=macd_param)
        data_ = signal.get_data(instrument)

        signal_threshold_buy = np.linspace(
            data_['macd'].quantile(1 - signal_threshold_quantile2),
            data_['macd'].quantile(1 - signal_threshold_quantile1), 
            num=steps['signal_threshold_buy']
        ).round(4)
        signal_threshold_sell = np.linspace(
            data_['macd'].quantile(signal_threshold_quantile2), 
            data_['macd'].quantile(signal_threshold_quantile1), 
            num=steps['signal_threshold_sell']
        ).round(4)

        slope_threshold_buy = [0, *data_.loc[data_['slope'] > 0, 'slope'].quantile(slope_threshold_quantiles).round(4)]
        slope_threshold_sell = [0, *data_.loc[data_['slope'] < 0, 'slope'].quantile([*reversed(slope_threshold_quantiles)]).round(4)]

        signal_dynamic_params_ = {
            'signal_threshold_buy': signal_threshold_buy,
            'signal_threshold_sell': signal_threshold_sell,
            'slope_threshold_buy': slope_threshold_buy,
            'slope_threshold_sell': slope_threshold_sell,
        }

        new_params = {
            **{k: [macd_param[k]] for k in macd_param},
            **signal_dynamic_params_,
            **strategy_params,
        }

        instrument_param_grids.append(ParameterGrid(new_params))
    return instrument_param_grids

In [40]:
rank_weight = pd.Series(
    {
        'std': -1,
        # 'var': 1,
        'cond_var': 1,
        'median': 1,
        'mean': 1,
        'sharpe_ratio': 1,
        'hit_ratio': 1,
        'drawdown': -1,
        'pnl': 1,
    }
)

In [47]:
def run_one(
    ticker,
    rank_weight,
    macd_param_grid,
    notional=1000,
    start_hist='10y',
    strategy_start_hist='1y',
    signal_threshold_quantile1=.75,
    signal_threshold_quantile2=.99,
    slope_threshold_quantiles=None,
    save_max=1000, 
    stop=None,
    stop2=None,
):
    c.USE_T_COST = True
    stats = {}
    strategies = {}
    instrument = FX(
        ticker=ticker,
        notional=notional,
        start_hist=start_hist
    )
    
    instrument_param_grids = make_param_grids(
        instrument=instrument,
        macd_param_grid=macd_param_grid,
        signal_threshold_quantile1=signal_threshold_quantile1,
        signal_threshold_quantile2=signal_threshold_quantile2,
        slope_threshold_quantiles=slope_threshold_quantiles,
        notional=notional,
        start_hist=start_hist,
    )
    stop = stop or len(instrument_param_grids)
    total = min(stop, len(instrument_param_grids)) * len(instrument_param_grids[0])
    
    for j, param in enumerate(chain.from_iterable(instrument_param_grids[:stop])):
        signal_params = {
            **{k: param[k] for k in macd_dynamic_params},
            'ti_kwargs': {k: param[k] for k in macd_params},
        }
        signal = MACDCustomSignal(**signal_params)

        custom_strategy_params = {k: param[k] for k in strategy_params}
        strategy = MACDCustomStrategy(
            instrument=instrument,
            signal=signal,
            start_hist=strategy_start_hist,
            **custom_strategy_params
        )
        strategy.run()
        if hasattr(strategy, '_summary'):
            strategies[j] = strategy
            stats[j] = strategy.stats
            if len(strategies) > save_max:
                stats_ = pd.DataFrame(stats).T
                ranks_ = (stats_ * rank_weight).rank().mean(axis=1).sort_values(ascending=False)
                to_drop = ranks_.index[-1]
                strategies.pop(to_drop)
                stats.pop(to_drop)
            print(
                f'{millify(j + 1)}/{millify(total)}', 
                ticker.upper(), 
                # {**signal_params, **custom_strategy_params}, 
                'PV',
                millify(strategy._summary['pv'].iloc[-1]),
                '    ',
                end="\r"
            )
        else:
            print('failed', strategy._failed, end='\r')
        if j == stop2:
            break
        
    stats = pd.DataFrame(stats).T 
    order = (stats * rank_weight).rank().mean(axis=1).sort_values(ascending=False).index
    stats = stats.loc[order, :]
    strategies = pd.Series(strategies).loc[order]
    return stats, strategies, instrument_param_grids, ticker

In [46]:
for t in fx:
    s = run_one(
        ticker=t, 
        macd_param_grid=macd_param_grid,
        rank_weight=rank_weight,
        stop2=10
    )[1][0]
    s._pnl

True/52.80k EURUSD PV 12.60     
True
True0/52.80k EURUSD PV -11.50     


In [48]:
res = Parallel(
    n_jobs=-1,
    verbose=100,
    # backend='multiprocessing'
)(
    delayed(run_one)(
        ticker=ticker, 
        macd_param_grid=macd_param_grid,
        rank_weight=rank_weight,
        stop2=None,
    ) for ticker in fx
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 42.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed: 42.2min finished


In [49]:
ress = {r[-1]: [*r[:-1]] for r in res}

In [50]:
ress.keys()

dict_keys(['EURUSD', 'USDJPY'])

In [64]:
ress['EURUSD'][1].iloc[0].instrument.quote

date
2013-01-11   1.33
2013-01-14   1.34
2013-01-15   1.33
2013-01-16   1.33
2013-01-17   1.34
             ... 
2023-01-05   1.05
2023-01-06   1.06
2023-01-09   1.07
2023-01-10   1.07
2023-01-11   1.07
Name: EURUSD quote, Length: 2609, dtype: float64

In [62]:
ress['EURUSD'][1].iloc[0]._pnl

trade  signal  entry_price  quote_chg  stop_loss  take_profit  \
date                                                                        
2022-03-14      1    1.00         1.10          0     1.994%       1.994%   
2022-03-15      1    1.00          NaN    -0.370%     1.994%       1.994%   
2022-03-16      1    1.00          NaN     0.460%     1.994%       1.994%   
2022-03-17      1    1.00          NaN     1.430%     0.608%       1.338%   
2022-05-04      2    1.00         1.05          0     1.026%       1.026%   
2022-05-05      2    1.00          NaN    -0.380%     1.026%       1.026%   
2022-05-06      2    1.00          NaN     0.400%     1.026%       1.026%   
2022-05-09      2    1.00          NaN     0.230%     1.026%       1.026%   
2022-05-10      2    1.00          NaN    -0.400%     1.026%       1.026%   
2022-05-16      3    1.00         1.04          0     1.613%       1.613%   
2022-05-17      3    1.00          NaN     1.180%     0.472%       1.038%   
2022-07-15      4    1.00         1.01          0     1.580%       1.580%   
2022-07-18      4    1.00          NaN     0.870%     1.580%       1.580%   
2022-07-19      4    1.00          NaN     0.730%     1.580%       1.580%   
2022-07-20      4    1.00          NaN    -0.490%     0.444%       0.977%   
2022-09-29      5    1.00      97.760%          0     1.972%       1.972%   
2022-09-30      5    1.00          NaN     0.260%     1.972%       1.972%   
2022-10-03      5    1.00          NaN     0.060%     1.972%       1.972%   
2022-10-04      5    1.00          NaN     1.620%     0.776%       1.707%   

            exit_price     pnl      pv  quote_chg_cum  
date                                                   
2022-03-14         NaN       0       0              0  
2022-03-15         NaN   -3.70   -3.70        -0.370%  
2022-03-16         NaN    4.60 90.000%         0.090%  
2022-03-17        1.11   14.30   15.20         1.520%  
2022-05-04         NaN       0   15.20              0  
2022-05-05         NaN   -3.80   11.40        -0.380%  
2022-05-06         NaN    4.00   15.40         0.020%  
2022-05-09         NaN    2.30   17.70         0.250%  
2022-05-10        1.05   -4.00   13.70        -0.150%  
2022-05-16         NaN       0   13.70              0  
2022-05-17        1.05   11.80   25.50         1.180%  
2022-07-15         NaN       0   25.50              0  
2022-07-18         NaN    8.70   34.20         0.870%  
2022-07-19         NaN    7.30   41.50         1.600%  
2022-07-20        1.02   -4.90   36.60         1.110%  
2022-09-29         NaN       0   36.60              0  
2022-09-30         NaN    2.60   39.20         0.260%  
2022-10-03         NaN 60.000%   39.80         0.320%  
2022-10-04     99.700%   16.20   56.00         1.940%

In [ ]:
ress['rx1'][1].iloc[0].instrument.t_cost

In [ ]:
ress['rx1'][1].iloc[0]._data.loc[ress['rx1'][1].iloc[0]._pnl.index, :]

In [ ]:
ress['rx1'][1].iloc[3]

In [ ]:
ress['rx1'][1]

In [ ]:
ress['rx1'][1].iloc[999]._pnl

In [ ]:
ress['rx1'][1].iloc[300]._pnl['pv'].plot()